# 1.

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import euclidean_distances
from sklearn_extra.cluster import KMedoids
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
import plotly.graph_objects as go

In [ ]:
#Cargar y verificar integridad de los datos. Realizar curación si corresponde.
data = pd.read_csv('Pokemon.csv')
data.head()


In [ ]:
#some values in type 2 are missing, so we will fill them with the value 'None'
data['Type 2'].fillna('None', inplace=True)

# 2.

In [ ]:
#Graficar matriz de correlación y realizar un scatter_matrix con plotly.

fig = px.scatter_matrix(data, dimensions=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'], color='Legendary')
fig.show()


In [ ]:
'''sns.pairplot(data, hue='Legendary')
plt.show()'''

In [ ]:
#total vs legendary, boxplot
fig = px.box(data, x='Legendary', y='Total')
fig.show()


In [ ]:
#total of only legendaries vs generation, scatter plot
fig = px.scatter(data, x='Generation', y='Total', color='Legendary')
#average line for total
data_common = data[data['Legendary'] == False]
fig.add_scatter(x=np.arange(1, 7), y=[data_common[data_common['Generation'] == i]['Total'].mean() for i in range(1, 7)], mode='lines', name='Average Total')
data_legendary = data[data['Legendary'] == True]
fig.add_scatter(x=np.arange(1, 7), y=[data_legendary[data_legendary['Generation'] == i]['Total'].mean() for i in range(1, 7)], mode='lines', name='Average Total Legendary')

fig.show()



# K-MEAN

In [ ]:
#K-means clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
n_types = data['Type 1'].nunique()
n_types

In [ ]:
data_copy = data.copy()
#data_copy = data_copy[data_copy['Legendary'] == False]
data_new = data_copy.drop(['#','Name', 'Generation', 'Legendary', 'Total', 'Type 2'], axis=1)#, 'Type 1', 'Type 2', 'Generation', 'Legendary', 'Total'], axis=1)

categorial_features = data_new.select_dtypes(include=['object']).columns
data_scaled = pd.get_dummies(data_new, columns=categorial_features)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_scaled)
categorial_features


In [ ]:

distances_list = []
sillhouette_list = []

K = 80
for j in range(2, K):
    kmeans = KMeans(n_clusters=j, random_state=42)
    data_new['Cluster'] = kmeans.fit_predict(data_scaled)
    
    total_distance = 0
    for i in range(j):
        cluster_points = data_scaled[data_new['Cluster'] == i]
        cluster_center = kmeans.cluster_centers_[i]
        total_distance += np.sum(np.linalg.norm(cluster_points - cluster_center, axis=1))

    
    #calculate sillhouette
    sillhouette_list.append(silhouette_score(data_scaled, data_new['Cluster']))
    
    mean_distance = total_distance / data.shape[0]
    distances_list.append(mean_distance)


In [ ]:
#plot elbow method

plt.plot(np.arange(2,K), distances_list)
plt.xlabel('Number of clusters')
plt.ylabel('Intracluster distance')
plt.xticks(np.arange(1,K,5))
plt.grid()
plt.title('Elbow method')
plt.show()


In [ ]:
#plot sillhouette
plt.plot(np.arange(2,K), sillhouette_list)
plt.xlabel('Number of clusters')
plt.ylabel('Intracluster distance')
plt.xticks(np.arange(1,K,5))
plt.grid()
plt.show()

In [ ]:
data_copy = data.copy()
#data_copy = data_copy[data_copy['Legendary'] == False]
data_new = data_copy.drop(['#','Name', 'Generation', 'Legendary', 'Total', 'Type 2'], axis=1)#, 'Type 1', 'Type 2', 'Generation', 'Legendary', 'Total'], axis=1)

categorial_features = data_new.select_dtypes(include=['object']).columns
data_scaled = pd.get_dummies(data_new, columns=categorial_features)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_scaled)
categorial_features

In [ ]:
# Set the number of clusters to 10
k = 18

# Perform KMeans clustering
kmeans = KMeans(n_clusters=k, random_state=42)
data_new['Cluster'] = kmeans.fit_predict(data_scaled)

In [ ]:
data_new['Cluster'].unique()

In [ ]:
#for each cluster, create a radar plot
# Set the number of clusters to 10
k = 18

# Perform KMeans clustering
kmeans = KMeans(n_clusters=k, random_state=42)
data_new['Cluster'] = kmeans.fit_predict(data_scaled)

clusters = data_new['Cluster'].unique()
for cluster in sorted(clusters):
    fig = go.Figure()
    cluster_data = data_new[data_new['Cluster'] == cluster].drop('Cluster', axis=1)
    numerical_features = cluster_data.select_dtypes(include=['float64', 'int64']).columns
    cluster_data = cluster_data[numerical_features]
    radar_data = cluster_data.mean().reset_index()
    radar_data.columns = ['Stats', 'Values']

    fig = px.line_polar(radar_data, r='Values', theta='Stats', line_close=True, range_r=[0, 200], title=f'Cluster {cluster}, {cluster_data.shape[0]} Pokemons')
    fig.show()

clusters
    


In [ ]:
#Realizar un grafico scatter_matrix con plotly usando como parámetro color la variable clústeres.
fig = plt.figure(figsize=(10, 15))
#add scatter matrix to the figure
fig = px.scatter_matrix(data_new, dimensions=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'], color='Cluster')
fig.update_layout(width=800, height=800)
fig.show()


In [ ]:
kmeans.cluster_centers_[:,:6]

In [ ]:
data_new.columns[1:-1]

In [ ]:
data_scaled[:,:6]

In [ ]:
temp = pd.DataFrame(data_scaled[:,:6], columns=data_new.columns[1:-1])
temp['Cluster'] = data_new['Cluster']
temp

In [ ]:
#sp attack vs hp, scatter plot
fig = px.scatter(temp, x='Sp. Atk', y='HP', color='Cluster')
#mark the cluster centers with a cross
for cluster in sorted(clusters):
    cluster_center = kmeans.cluster_centers_[cluster]
    fig.add_scatter(x=[cluster_center[3]], y=[cluster_center[0]], mode='markers', marker=dict(size=10, symbol='x', color='#0ce7f8'), name=f'Cluster {cluster} center')
fig.show()

# K-MEDIOD

In [ ]:
data_copy = data.copy()
#data_copy = data_copy[data_copy['Legendary'] == False]
data_new = data_copy.drop(['#','Name', 'Generation', 'Legendary', 'Total', 'Type 2'], axis=1)#, 'Type 1', 'Type 2', 'Generation', 'Legendary', 'Total'], axis=1)

categorial_features = data_new.select_dtypes(include=['object']).columns
data_scaled = pd.get_dummies(data_new, columns=categorial_features)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_scaled)
categorial_features

In [ ]:
K = 40

distances_list = []
sillhouette_list = []

for j in range(2, K):
    kmediod = KMedoids(n_clusters=j, random_state=42)
    data_new['Cluster'] = kmediod.fit_predict(data_scaled)
    
    total_distance = 0
    for i in range(j):
        cluster_points = data_scaled[data_new['Cluster'] == i]
        cluster_center = kmediod.cluster_centers_[i]
        total_distance += np.sum(np.linalg.norm(cluster_points - cluster_center, axis=1))**2

    
    #calculate sillhouette
    sillhouette_list.append(silhouette_score(data_scaled, data_new['Cluster']))
    
    mean_distance = total_distance / data_new.shape[0]
    distances_list.append(mean_distance)

In [ ]:
#plot elbow method
plt.plot(np.arange(2,K), distances_list)
plt.xlabel('Number of clusters')
plt.ylabel('Intracluster distance')
plt.title('Elbow method')
plt.show()


In [ ]:
#plot sillhouette
plt.plot(np.arange(2,K), sillhouette_list)
plt.xlabel('Number of clusters')
plt.ylabel('sillhouette value')
plt.xticks(np.arange(1,K,K//10))
plt.grid()
plt.show()

In [ ]:
data_copy = data.copy()
#data_copy = data_copy[data_copy['Legendary'] == False]
data_new = data_copy.drop(['#','Name', 'Generation', 'Legendary', 'Total', 'Type 2'], axis=1)#, 'Type 1', 'Type 2', 'Generation', 'Legendary', 'Total'], axis=1)

categorial_features = data_new.select_dtypes(include=['object']).columns
data_scaled = pd.get_dummies(data_new, columns=categorial_features)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_scaled)
categorial_features

In [ ]:
k = 10

# Perform KMeans clustering
kmediods = KMedoids(n_clusters=k, random_state=42)
data_new['Cluster'] = kmediods.fit_predict(data_scaled)

clusters = data_new['Cluster'].unique()
for cluster in sorted(clusters):
    fig = go.Figure()
    cluster_data = data_new[data_new['Cluster'] == cluster].drop('Cluster', axis=1)
    numerical_features = cluster_data.select_dtypes(include=['float64', 'int64']).columns
    cluster_data = cluster_data[numerical_features]
    radar_data = cluster_data.mean().reset_index()
    radar_data.columns = ['Stats', 'Values']

    fig = px.line_polar(radar_data, r='Values', theta='Stats', line_close=True, range_r=[0, 200], title=f'Cluster {cluster}, {cluster_data.shape[0]} Pokemons')
    fig.show()

clusters

In [ ]:
#Realizar un grafico scatter_matrix con plotly usando como parámetro color la variable clústeres.
fig = plt.figure(figsize=(10, 15))
#add scatter matrix to the figure
fig = px.scatter_matrix(data_new, dimensions=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'], color='Cluster')
fig.update_layout(width=800, height=800)
fig.show()

In [ ]:
temp = pd.DataFrame(data_scaled[:,:6], columns=data_new.columns[1:-1])
temp['Cluster'] = data_new['Cluster']
temp

In [67]:
#sp attack vs hp, scatter plot
fig = px.scatter(temp, x='Sp. Atk', y='HP', color='Cluster')
#mark the cluster centers with a cross
for cluster in sorted(clusters):
    cluster_center = kmediods.cluster_centers_[cluster]
    fig.add_scatter(x=[cluster_center[3]], y=[cluster_center[0]], mode='markers', marker=dict(size=10, symbol='x', color='#0ce7f8'), name=f'Cluster {cluster} center')
fig.show()

# K-MEDIAN

In [ ]:
data_copy = data.copy()
#data_copy = data_copy[data_copy['Legendary'] == False]
data_new = data_copy.drop(['#','Name', 'Generation', 'Legendary', 'Total', 'Type 2'], axis=1)#, 'Type 1', 'Type 2', 'Generation', 'Legendary', 'Total'], axis=1)

categorial_features = data_new.select_dtypes(include=['object']).columns
data_scaled = pd.get_dummies(data_new, columns=categorial_features)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_scaled)
categorial_features

In [ ]:
K = 80

distances_list = []
sillhouette_list = []

for j in range(2, K):
    kmediod = KMedoids(n_clusters=j, random_state=42)
    data_new['Cluster'] = kmediod.fit_predict(data_scaled)
    
    total_distance = 0
    for i in range(j):
        cluster_points = data_scaled[data_new['Cluster'] == i]
        cluster_center = kmediod.cluster_centers_[i]
        total_distance += np.sum(np.linalg.norm(cluster_points - cluster_center, axis=1))

    
    #calculate sillhouette
    sillhouette_list.append(silhouette_score(data_scaled, data_new['Cluster']))
    
    mean_distance = total_distance / data_new.shape[0]
    distances_list.append(mean_distance)

In [ ]:
#plot elbow method
plt.plot(np.arange(2,K), distances_list)
plt.xlabel('Number of clusters')
plt.ylabel('Intracluster distance')
plt.title('Elbow method')
plt.show()

In [ ]:
#plot sillhouette
plt.plot(np.arange(2,K), sillhouette_list)
plt.xlabel('Number of clusters')
plt.ylabel('sillhouette value')
plt.xticks(np.arange(1,K,5))
plt.grid()
plt.show()

In [ ]:
data_copy = data.copy()
#data_copy = data_copy[data_copy['Legendary'] == False]
data_new = data_copy.drop(['#','Name', 'Generation', 'Legendary', 'Total', 'Type 2'], axis=1)#, 'Type 1', 'Type 2', 'Generation', 'Legendary', 'Total'], axis=1)

categorial_features = data_new.select_dtypes(include=['object']).columns
data_scaled = pd.get_dummies(data_new, columns=categorial_features)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_scaled)
categorial_features

In [ ]:
k = 20

# Perform KMeans clustering
kmediods = KMedoids(n_clusters=k, random_state=42)
data_new['Cluster'] = kmediods.fit_predict(data_scaled)

clusters = data_new['Cluster'].unique()
for cluster in sorted(clusters):
    fig = go.Figure()
    cluster_data = data_new[data_new['Cluster'] == cluster].drop('Cluster', axis=1)
    numerical_features = cluster_data.select_dtypes(include=['float64', 'int64']).columns
    cluster_data = cluster_data[numerical_features]
    radar_data = cluster_data.mean().reset_index()
    radar_data.columns = ['Stats', 'Values']

    fig = px.line_polar(radar_data, r='Values', theta='Stats', line_close=True, range_r=[0, 200], title=f'Cluster {cluster}, {cluster_data.shape[0]} Pokemons')
    fig.show()

clusters

In [64]:
#Realizar un grafico scatter_matrix con plotly usando como parámetro color la variable clústeres.
fig = plt.figure(figsize=(10, 15))
#add scatter matrix to the figure
fig = px.scatter_matrix(data_new, dimensions=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'], color='Cluster')
fig.update_layout(width=800, height=800)
fig.show()

<Figure size 1000x1500 with 0 Axes>

In [65]:
temp = pd.DataFrame(data_scaled[:,:6], columns=data_new.columns[1:-1])
temp['Cluster'] = data_new['Cluster']
temp

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Cluster
0,0.173228,0.237838,0.195556,0.298913,0.214286,0.228571,7
1,0.232283,0.308108,0.257778,0.380435,0.285714,0.314286,7
2,0.311024,0.416216,0.346667,0.489130,0.380952,0.428571,6
3,0.311024,0.513514,0.524444,0.608696,0.476190,0.428571,6
4,0.149606,0.254054,0.168889,0.271739,0.142857,0.342857,11
...,...,...,...,...,...,...,...
795,0.192913,0.513514,0.644444,0.489130,0.619048,0.257143,13
796,0.192913,0.837838,0.466667,0.815217,0.428571,0.600000,13
797,0.311024,0.567568,0.244444,0.760870,0.523810,0.371429,15
798,0.311024,0.837838,0.244444,0.869565,0.523810,0.428571,15


In [68]:
#sp attack vs hp, scatter plot
fig = px.scatter(temp, x='Sp. Atk', y='HP', color='Cluster')
#mark the cluster centers with a cross
for cluster in sorted(clusters):
    cluster_center = kmediods.cluster_centers_[cluster]
    fig.add_scatter(x=[cluster_center[3]], y=[cluster_center[0]], mode='markers', marker=dict(size=10, symbol='x', color='#0ce7f8'), name=f'Cluster {cluster} center')
fig.show()